In [1]:
!pip install pyspark pandas sqlalchemy psycopg2-binary

  Using cached pyspark-4.0.1-py2.py3-none-any.whl
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached sqlalchemy-2.0.44-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
  Using cached psycopg2_binary-2.9.11-cp313-cp313-win_amd64.whl.metadata (5.1 kB)
  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached numpy-2.3.4-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached greenlet-3.2.4-cp313-cp313-win_amd64.whl.metadata (4.2 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl (11.0 MB)
Using cached sqlalchemy-2.0.44-cp313-cp313-win_amd64.whl (2.1 MB)
Using cached psycopg2_binary-2.9.11-cp313-cp313-win_amd64.whl (2.7 MB)
Using cached greenlet-3.2.4-cp313-cp313-win_

In [2]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 4.0.1
      /_/
                        
Using Scala version 2.13.16, Java HotSpot(TM) 64-Bit Server VM, 21.0.9
Branch HEAD
Compiled by user runner on 2025-09-02T03:10:51Z
Revision 29434ea766b0fc3c3bf6eaadb43a8f931133649e
Url https://github.com/apache/spark
Type --help for more information.


In [ ]:
# Initialise our Spark Session
spark = SparkSession.builder.appName('DealWithUsApp').getOrCreate()